In [5]:
import boto3
import time

# Replace these with your desired names
log_group_name = 'BGLAlerts'
log_stream_name = 'FrequentPatternWarnings'

# Initialize CloudWatch Logs client
client = boto3.client('logs', region_name='us-east-1')  # Change to your region

# Step 1: Create log group (if not exists)
try:
    client.create_log_group(logGroupName=log_group_name)
except client.exceptions.ResourceAlreadyExistsException:
    pass

# Step 2: Create log stream (if not exists)
try:
    client.create_log_stream(logGroupName=log_group_name, logStreamName=log_stream_name)
except client.exceptions.ResourceAlreadyExistsException:
    pass

# Step 3: Send a test log event
def send_log_event(message):
    # Get the sequence token (required for subsequent events)
    streams = client.describe_log_streams(logGroupName=log_group_name, logStreamNamePrefix=log_stream_name)
    upload_sequence_token = streams['logStreams'][0].get('uploadSequenceToken')

    kwargs = {
        'logGroupName': log_group_name,
        'logStreamName': log_stream_name,
        'logEvents': [{
            'timestamp': int(time.time() * 1000),
            'message': message
        }]
    }

    if upload_sequence_token:
        kwargs['sequenceToken'] = upload_sequence_token

    response = client.put_log_events(**kwargs)
    print(response)
    print("✅ Message pushed to CloudWatch Logs.")

# Send test message
send_log_event("Test Alert: Pattern T1 → T2 detected with 85% failure likelihood.")


{'nextSequenceToken': '49662280640771539065123775408644733161545247012081893522', 'ResponseMetadata': {'RequestId': '6a9898ff-260d-42a8-b88a-bbac502fb9bd', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '6a9898ff-260d-42a8-b88a-bbac502fb9bd', 'content-type': 'application/x-amz-json-1.1', 'content-length': '80', 'date': 'Sat, 03 May 2025 20:19:32 GMT'}, 'RetryAttempts': 0}}
✅ Message pushed to CloudWatch Logs.
